In [17]:
import pandas as pd
import sqlite3
import seaborn as sb
import matplotlib as plt

In [18]:
# SQLite database passage
conn = sqlite3.connect('space_missions.sqlite')
cur = conn.cursor()

tablename = 'records'

df = pd.read_csv('space_missions_normalized.csv')
df.to_sql(tablename, conn, if_exists='replace', index=False)

4630

In [19]:
# First analysis query
# List of companies per country
q = '''
SELECT Country, COUNT(DISTINCT Company) AS TotalCompaniesPerCountry
FROM records
GROUP BY Country
ORDER BY TotalCompaniesPerCountry DESC;
'''
# pd.read_sql_query(q,conn)
pd.read_sql_query(q,conn)


,Country,TotalCompaniesPerCountry
0,USA,19
1,Kazakhstan,11
2,Russia,9
3,China,8
4,France,5
5,Japan,4
6,Australia,3
7,Iran,2
8,Yellow Sea,1
9,South Korea,1


In [20]:
# Creating a new database to save the insights
conn2 = sqlite3.connect('space_missions_insights.sqlite') 
cur = conn.cursor()

NtableName = 'companies_country'
df2 = pd.read_sql_query(q,conn)
df2.to_sql(NtableName, conn2, if_exists='replace', index=False) 

22

In [21]:
# Top 5 countries with rocket launches
q2 = '''
SELECT Country, COUNT(*) AS TotalLaunches
FROM records
GROUP BY Country
ORDER BY TotalLaunches DESC
LIMIT 5;
'''
NtableName2='total_launches_country'
df3 = pd.read_sql_query(q2, conn)
df3.to_sql(NtableName2, conn2, if_exists='replace', index=False)


print(df3)

      Country  TotalLaunches
0         USA           1467
1      Russia           1416
2  Kazakhstan            719
3       China            365
4      France            318


In [22]:
# Top 3 with success launches
q3 = '''
SELECT Country, COUNT(*) AS SuccesfullLaunches
FROM records
WHERE MissionStatus = 'Success'
GROUP BY Country
ORDER BY SuccesfullLaunches DESC
LIMIT 3;
'''
NtableName3 = 'top_suc-mission'
df4 = pd.read_sql_query(q3, conn)
df4.to_sql(NtableName3, conn2, if_exists='replace', index=False)

print(df4)

      Country  SuccesfullLaunches
0      Russia                1323
1         USA                1298
2  Kazakhstan                 625


In [23]:
# Top 3 with unsuccessful launches
q4 = '''
SELECT Country, COUNT(*) AS UnsuccessfullLaunches
FROM records
WHERE MissionStatus = 'Failure' OR MissionStatus = 'PartialFailure' OR MissionStatus = 'Prelaunch Failure'
GROUP BY Country
ORDER BY UnsuccessfullLaunches DESC
LIMIT 3;
'''
NtableName4 = 'top_uns-mission'
df5 = pd.read_sql_query(q4, conn)
df5.to_sql(NtableName4, conn2, if_exists='replace', index=False)

print(df5)

      Country  UnsuccessfullLaunches
0         USA                    135
1  Kazakhstan                     73
2      Russia                     63


In [24]:
# Succesfull & Unsuccessfull launches ratio (TOP 10)
q5 = '''
SELECT
    Country,
    SUM(CASE WHEN MissionStatus = 'Success' THEN 1 ELSE 0 END) AS SuccessfulLaunches,
    SUM(CASE WHEN MissionStatus IN ('Failure', 'partialfailure', 'Prelaunchfailure') THEN 1 ELSE 0 END) AS UnsuccessfulLaunches,
    (SUM(CASE WHEN MissionStatus = 'Success' THEN 1 ELSE 0 END) * 1.0 / NULLIF(SUM(CASE WHEN MissionStatus = 'Failure' THEN 1 ELSE 0 END), 0)) AS SuccessToFailureRatio
FROM
    records
GROUP BY
    Country
ORDER BY
    SuccessToFailureRatio DESC
LIMIT 10;
'''

NtableName4 = 'ratios'
df6 = pd.read_sql_query(q5,conn)
df6.to_sql(NtableName4, conn2, if_exists='replace', index=False)

print(df6)

         Country  SuccessfulLaunches  UnsuccessfulLaunches  \
0         France                 299                    14   
1         Russia                1323                    63   
2          China                 335                    24   
3          Japan                 117                    10   
4  Pacific Ocean                  33                     3   
5            USA                1298                   134   
6     Kazakhstan                 625                    72   
7    New Zealand                  25                     3   
8          India                  68                     9   
9         Israel                   9                     2   

   SuccessToFailureRatio  
0              21.357143  
1              21.000000  
2              13.958333  
3              11.700000  
4              11.000000  
5               9.686567  
6               8.680556  
7               8.333333  
8               7.555556  
9               4.500000  
